In [1]:
from tqdm.notebook import tqdm

import numpy as np

import cv2

import matplotlib.pyplot as plt
from matplotlib.animation import FuncAnimation
from IPython.display import display, HTML, Video

import math

import warnings

# Suprimir todas las advertencias
warnings.simplefilter("ignore")

In [2]:
# # Función de paleta en la CPU
# def palette(t):
#     a = np.array([0.5, 0.5, 0.5])
#     b = np.array([0.5, 0.5, 0.5])
#     c = np.array([1.0, 1.0, 1.0])
#     d = np.array([0.263, 0.416, 0.557])
    
#     return a + b * np.cos(6.28318 * (c * t + d))

In [3]:
def palette(t):
    magenta = np.array([0.91, 0.12, 0.39])  # Magenta (superior)
    purple = np.array([0.53, 0.19, 0.51])   # Púrpura (medio)
    blue = np.array([0.04, 0.31, 0.58])     # Azul (inferior)
    
    # Interpolar entre los colores basados en el valor de t
    if t < 0.5:
        return magenta + (purple - magenta) * (t * 2)
    else:
        return purple + (blue - purple) * ((t - 0.5) * 2)

In [4]:
def vectorized_palette(t):
    magenta = np.array([0.91, 0.12, 0.39])  # Magenta (superior)
    purple = np.array([0.53, 0.19, 0.51])   # Púrpura (medio)
    blue = np.array([0.04, 0.31, 0.58])     # Azul (inferior)
    
    t_shape = t.shape
    t = t.ravel()
    
    result = np.empty((t.size, 3))
    mask = t < 0.5
    result[mask] = magenta + (purple - magenta) * (t[mask, None] * 2)
    result[~mask] = purple + (blue - purple) * ((t[~mask, None] - 0.5) * 2)
    
    return result.reshape(*t_shape, 3)

In [5]:
# Función principal que se ejecutará en la CPU
def render(pixels, iTime):
    for x in tqdm(range(WIDTH), desc="Rendering", leave=False):
        for y in range(HEIGHT):
            uv = np.array([(x * 2.0 - WIDTH) / HEIGHT, (y * 2.0 - HEIGHT) / HEIGHT])
            uv0 = uv.copy()
            finalColor = np.array([0.0, 0.0, 0.0])

            for i in range(4):
                uv = (uv * 1.5) % 1 - 0.5
                d = np.linalg.norm(uv) * np.exp(-np.linalg.norm(uv0))
                col = palette(np.linalg.norm(uv0) + i * 0.4 + iTime * 0.4)
                d = abs(np.sin(d * 8.0 + iTime) / 8.0)
                d = pow(0.01 / d, 1.2)

                finalColor += col * d

            pixels[x, y] = (finalColor * 255).astype(np.uint8)
    return pixels

In [6]:
# Función de actualización para la animación
def update(frame):
    global pixels, iTime
    iTime += 0.05
    render(pixels, iTime)
    im.set_array(pixels)
    return im,

In [7]:
fps = 30
seconds = 15
fourcc = cv2.VideoWriter_fourcc(*"XVID")
WIDTH, HEIGHT = 800, 800
# pixels = np.zeros((WIDTH, HEIGHT, 3), dtype=np.uint8)

In [8]:
# # Configuración inicial para la animación
# fig, ax = plt.subplots(figsize=(8, 8))
# pixels = np.zeros((WIDTH, HEIGHT, 3), dtype=np.uint8)
# iTime = 0.2
# render(pixels, iTime)
# im = plt.imshow(pixels, animated=True)

# ani = FuncAnimation(fig, update, frames=100, blit=True, interval=50)

# # Guardar la animación como un archivo .mp4
# ani.save("animation.mp4", writer="ffmpeg", fps=20)

# plt.close()

In [9]:
# # Para mostrar en el notebook
# HTML(ani.to_jshtml())

In [10]:
# import dask.array as da

# def parallel_vectorized_render(pixels, iTime):
#     # Create coordinate grids
#     x = da.linspace(0, WIDTH - 1, WIDTH)
#     y = da.linspace(0, HEIGHT - 1, HEIGHT)
#     xv, yv = da.meshgrid(x, y, sparse=False, indexing='ij')
    
#     # Calculate uv values
#     uv_x = (xv * 2.0 - WIDTH) / HEIGHT
#     uv_y = (yv * 2.0 - HEIGHT) / HEIGHT
#     uv = da.stack((uv_x, uv_y), axis=-1)
#     uv0 = uv.copy()
    
#     # Initialize finalColor array
#     finalColor = da.zeros((WIDTH, HEIGHT, 3))
    
#     for i in range(4):
#         uv = (uv * 1.5) % 1 - 0.5
#         d = da.linalg.norm(uv, axis=-1) * da.exp(-da.linalg.norm(uv0, axis=-1))
#         col = vectorized_palette(da.linalg.norm(uv0, axis=-1) + i * 0.4 + iTime * 0.4).compute()
#         d = da.abs(da.sin(d * 8.0 + iTime) / 8.0)
#         d = da.power(0.01 / d, 1.2).reshape(WIDTH, HEIGHT, 1)
#         finalColor += col * d
        
#     pixels[:, :] = (finalColor * 255).astype(np.uint8).compute()
#     return pixels

# # Test the parallel vectorized render function
# parallel_vectorized_render(pixels_test, iTime_test)

In [11]:
# from numba import jit, prange

# @jit(nopython=True, parallel=True)
# def numba_vectorized_render(pixels, iTime):
#     # Create coordinate grids
#     x = np.linspace(0, WIDTH - 1, WIDTH)
#     y = np.linspace(0, HEIGHT - 1, HEIGHT)
#     xv, yv = np.meshgrid(x, y, sparse=False, indexing='ij')
    
#     # Calculate uv values
#     uv_x = (xv * 2.0 - WIDTH) / HEIGHT
#     uv_y = (yv * 2.0 - HEIGHT) / HEIGHT
#     uv = np.stack((uv_x, uv_y), axis=-1)
#     uv0 = uv.copy()
    
#     # Initialize finalColor array
#     finalColor = np.zeros((WIDTH, HEIGHT, 3))
    
#     for i in range(4):
#         uv = (uv * 1.5) % 1 - 0.5
#         d = np.linalg.norm(uv, axis=-1) * np.exp(-np.linalg.norm(uv0, axis=-1))
#         col = vectorized_palette(np.linalg.norm(uv0, axis=-1) + i * 0.4 + iTime * 0.4)
#         d = np.abs(np.sin(d * 8.0 + iTime) / 8.0)
#         d = np.power(0.01 / d, 1.2).reshape(WIDTH, HEIGHT, 1)
#         finalColor += col * d
        
#     pixels[:, :] = (finalColor * 255).astype(np.uint8)
#     return pixels

# # Test the numba vectorized render function
# numba_vectorized_render(pixels_test, iTime_test)

In [12]:
# def vectorized_render(pixels, iTime):
#     # Create coordinate grids
#     x = np.linspace(0, WIDTH - 1, WIDTH)
#     y = np.linspace(0, HEIGHT - 1, HEIGHT)
#     xv, yv = np.meshgrid(x, y, sparse=False, indexing='ij')
    
#     # Calculate uv values
#     uv_x = (xv * 2.0 - WIDTH) / HEIGHT
#     uv_y = (yv * 2.0 - HEIGHT) / HEIGHT
#     uv = np.stack((uv_x, uv_y), axis=-1)
#     uv0 = uv.copy()
    
#     # Initialize finalColor array
#     finalColor = np.zeros((WIDTH, HEIGHT, 3))
    
#     for i in range(4):
#         uv = (uv * 1.5) % 1 - 0.5
#         d = np.linalg.norm(uv, axis=-1) * np.exp(-np.linalg.norm(uv0, axis=-1))
#         col = palette(np.linalg.norm(uv0, axis=-1) + i * 0.4 + iTime * 0.4)
#         d = np.abs(np.sin(d * 8.0 + iTime) / 8.0)
#         d = np.power(0.01 / d, 1.2).reshape(WIDTH, HEIGHT, 1)
#         finalColor += col * d
        
#     pixels[:, :] = (finalColor * 255).astype(np.uint8)
#     return pixelsb

In [13]:
def optimized_render(pixels, iTime):
    # Same as vectorized_render but uses vectorized_palette
    # Create coordinate grids
    x = np.linspace(0, WIDTH - 1, WIDTH)
    y = np.linspace(0, HEIGHT - 1, HEIGHT)
    xv, yv = np.meshgrid(x, y, sparse=False, indexing='ij')
    
    # Calculate uv values
    uv_x = (xv * 2.0 - WIDTH) / HEIGHT
    uv_y = (yv * 2.0 - HEIGHT) / HEIGHT
    uv = np.stack((uv_x, uv_y), axis=-1)
    uv0 = uv.copy()
    
    # Initialize finalColor array
    finalColor = np.zeros((WIDTH, HEIGHT, 3))
    
    for i in range(4):
        uv = (uv * 1.5) % 1 - 0.5
        d = np.linalg.norm(uv, axis=-1) * np.exp(-np.linalg.norm(uv0, axis=-1))
        col = vectorized_palette(np.linalg.norm(uv0, axis=-1) + i * 0.4 + iTime * 0.4)
        d = np.abs(np.sin(d * 8.0 + iTime) / 8.0)
        d = np.power(0.01 / d, 1.2).reshape(WIDTH, HEIGHT, 1)
        finalColor += col * d
        
    pixels[:, :] = (finalColor * 255).astype(np.uint8)
    return pixels

In [14]:
# pixels_test = np.zeros((WIDTH, HEIGHT, 3), dtype=np.uint8)
# iTime_test = 0.2

# vectorized_render(pixels_test, iTime_test)

In [15]:
# Create video writer object
video_writer = cv2.VideoWriter('movie.mp4', fourcc, fps, (WIDTH, HEIGHT))

OpenCV: FFMPEG: tag 0x44495658/'XVID' is not supported with codec id 12 and format 'mp4 / MP4 (MPEG-4 Part 14)'
OpenCV: FFMPEG: fallback to use tag 0x7634706d/'mp4v'


In [16]:
# Generate video frames using the optimized_render function
for frame in range(fps * seconds):
    iTime = frame / fps
    pixels = np.zeros((WIDTH, HEIGHT, 3), dtype=np.uint8)
    optimized_render(pixels, iTime)
    video_writer.write(pixels)

# Release video writer object
video_writer.release()

In [17]:
# Display the video in the notebook
display(Video('movie.mp4'))